## En esta segunda etapa evaluamos cuantitativamente el rendimiento del algoritmo Multinomial Naive Bayes sobre un corpus de reseñas de películas en español, comparando distintas técnicas de limpieza y normalización de texto (Stemming, Lematización, StopWords y sus combinaciones).

In [ ]:
!pip install -q spacy
!python -m spacy download es_core_news_sm
!pip install pandas numpy nltk spacy scikit-learn unidecode
!python -m nltk.downloader punkt stopwords
!pip install --upgrade gdown

# 1. Importación de bibliotecas necesarias

In [3]:
import pandas as pd
import numpy as np
import re
import unicodedata
import nltk
import spacy
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

#from google.colab import drive
from IPython.display import display
from collections import Counter
from IPython.display import display, HTML

In [2]:
import gdown

# ID del archivo de Google Drive
file_id = '1XYmvG4gfXYgZzT_MG_Nn-GRWNYQUSbqF'

# URL directa
url = f'https://drive.google.com/uc?id={file_id}'

# Nombre de salida que usabas en tu código
output = 'IMDBDatasetSPANISH.csv'

# Descarga del archivo
gdown.download(url, output, quiet=False)

# Lectura del dataset
import pandas as pd
df = pd.read_csv(output)
df.head()


Downloading...
From (original): https://drive.google.com/uc?id=1XYmvG4gfXYgZzT_MG_Nn-GRWNYQUSbqF
From (redirected): https://drive.google.com/uc?id=1XYmvG4gfXYgZzT_MG_Nn-GRWNYQUSbqF&confirm=t&uuid=3df55ab7-2ce9-40f5-8fee-672b642cda37
To: /content/IMDBDatasetSPANISH.csv
100%|██████████| 137M/137M [00:01<00:00, 71.4MB/s]


,Unnamed: 0,review_en,review_es,sentiment,sentimiento
0,0,One of the other reviewers has mentioned that ...,Uno de los otros críticos ha mencionado que de...,positive,positivo
1,1,A wonderful little production. The filming tec...,Una pequeña pequeña producción.La técnica de f...,positive,positivo
2,2,I thought this was a wonderful way to spend ti...,Pensé que esta era una manera maravillosa de p...,positive,positivo
3,3,Basically there's a family where a little boy ...,"Básicamente, hay una familia donde un niño peq...",negative,negativo
4,4,"Petter Mattei's ""Love in the Time of Money"" is...","El ""amor en el tiempo"" de Petter Mattei es una...",positive,positivo


In [ ]:
if df is not None:
    # Visualizar las primeras filas del dataset
    print("## Primeras 5 filas del dataset:")
    display(df.head())

    # Identificar las columnas disponibles
    print("\n## Columnas del dataset:")
    display(df.columns)

    # Obtener información general sobre el dataset (tipos de datos, valores no nulos, etc.)
    print("\n## Información general del dataset:")
    display(df.info())

    # Estadísticas descriptivas (si aplica para columnas numéricas)
    print("\n## Estadísticas descriptivas:")
    display(df.describe(include='all'))

    # Familiarizarse con el tipo de texto y la estructura
    # Para este dataset de IMDB, las columnas relevantes son 'review_es' y 'sentimiento'.
    # Vamos a ver algunos ejemplos de reseñas y sus sentimientos.
    if 'review_es' in df.columns and 'sentimiento' in df.columns:
        print("\n## Ejemplos de reseñas y sentimientos:")
        # Seleccionamos algunas filas aleatorias para ver ejemplos
        display(df[['review_es', 'sentimiento']].sample(5))
    else:
        print("\nLas columnas 'review_es' y/o 'sentimiento' no se encontraron en el dataset.")
        print("Por favor, verifica los nombres de las columnas en tu archivo CSV.")

## Primeras 5 filas del dataset:


,Unnamed: 0,review_en,review_es,sentiment,sentimiento
0,0,One of the other reviewers has mentioned that ...,Uno de los otros críticos ha mencionado que de...,positive,positivo
1,1,A wonderful little production. The filming tec...,Una pequeña pequeña producción.La técnica de f...,positive,positivo
2,2,I thought this was a wonderful way to spend ti...,Pensé que esta era una manera maravillosa de p...,positive,positivo
3,3,Basically there's a family where a little boy ...,"Básicamente, hay una familia donde un niño peq...",negative,negativo
4,4,"Petter Mattei's ""Love in the Time of Money"" is...","El ""amor en el tiempo"" de Petter Mattei es una...",positive,positivo



## Columnas del dataset:


Index(['Unnamed: 0', 'review_en', 'review_es', 'sentiment', 'sentimiento'], dtype='object')


## Información general del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   50000 non-null  int64 
 1   review_en    50000 non-null  object
 2   review_es    50000 non-null  object
 3   sentiment    50000 non-null  object
 4   sentimiento  50000 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.9+ MB


None


## Estadísticas descriptivas:


,Unnamed: 0,review_en,review_es,sentiment,sentimiento
count,50000.000000,50000,50000,50000,50000
unique,NaN,49581,49599,2,2
top,NaN,Loved today's show!!! It was a variety and not...,"Hilarante, limpio, alegre y digno de cita.¿Qué...",positive,positivo
freq,NaN,5,4,25000,25000
mean,24999.500000,NaN,NaN,NaN,NaN
std,14433.901067,NaN,NaN,NaN,NaN
min,0.000000,NaN,NaN,NaN,NaN
25%,12499.750000,NaN,NaN,NaN,NaN
50%,24999.500000,NaN,NaN,NaN,NaN
75%,37499.250000,NaN,NaN,NaN,NaN



## Ejemplos de reseñas y sentimientos:


,review_es,sentimiento
46223,Se debe felicitar a Bashki por intentar conver...,negativo
5497,Esta película es genial. Como se escucha a men...,positivo
41950,Parker (Johnathan Schaech) es un aspirante a e...,positivo
30518,Caracteres planos que no lo haces y nunca te i...,negativo
23041,"Esta película es bastante buena, en realidad e...",positivo


In [ ]:
if df is not None and 'review_es' in df.columns:
    # Definir el patrón de regex para encontrar signos de puntuación y caracteres especiales.
    # [\W_] coincide con cualquier carácter que NO sea una palabra (letras, números, guion bajo).
    # También podrías definir un conjunto específico de caracteres a eliminar,
    # por ejemplo, r'[.,!?;:"\'()\[\]{}<>/@#$%^&*+=-_`~|]' para ser más específico.
    pattern = r'[\W_]'

    # Aplicar la limpieza a la columna 'review_es'.
    # Usamos una función lambda y re.sub para reemplazar las coincidencias con un espacio.
    # .str.replace() en pandas también es una opción, pero re.sub da más flexibilidad con regex.
    df['review_es_cleaned'] = df['review_es'].apply(lambda x: re.sub(pattern, ' ', x) if isinstance(x, str) else x)

    # Mostrar las primeras filas para ver el resultado de la limpieza
    print("## Primeras 5 filas del dataset con la columna limpia ('review_es_cleaned'):")
    display(df[['review_es', 'review_es_cleaned']].head())

    # Mostrar algunos ejemplos aleatorios para verificar la limpieza
    print("\n## Ejemplos aleatorios de reseñas antes y después de la limpieza:")
    display(df[['review_es', 'review_es_cleaned']].sample(5))

else:
    print("El DataFrame no se cargó correctamente o la columna 'review_es' no existe.")
    print("Por favor, revisa el Paso 1 y asegúrate de que el archivo CSV se cargue y contenga la columna 'review_es'.")

## Primeras 5 filas del dataset con la columna limpia ('review_es_cleaned'):


,review_es,review_es_cleaned
0,Uno de los otros críticos ha mencionado que de...,Uno de los otros críticos ha mencionado que de...
1,Una pequeña pequeña producción.La técnica de f...,Una pequeña pequeña producción La técnica de f...
2,Pensé que esta era una manera maravillosa de p...,Pensé que esta era una manera maravillosa de p...
3,"Básicamente, hay una familia donde un niño peq...",Básicamente hay una familia donde un niño peq...
4,"El ""amor en el tiempo"" de Petter Mattei es una...",El amor en el tiempo de Petter Mattei es una...



## Ejemplos aleatorios de reseñas antes y después de la limpieza:


,review_es,review_es_cleaned
2383,"""Steve"" (Chris Hoffman) reúne a un grupo de la...",Steve Chris Hoffman reúne a un grupo de la...
20488,¡Conoces la historia ... sólo niños en el bosq...,Conoces la historia sólo niños en el bosq...
7394,Primero déjame ser honesto.No vi toda esta pel...,Primero déjame ser honesto No vi toda esta pel...
36983,Acabo de ver esta película en la fantasía Film...,Acabo de ver esta película en la fantasía Film...
38137,"Una vez más, estaba navegando por el contenedo...",Una vez más estaba navegando por el contenedo...


In [ ]:
if df is not None and 'review_es_cleaned' in df.columns:
    # Aplicar la conversión a minúsculas a la columna limpia.
    # El método .str.lower() de pandas es eficiente para columnas de tipo string.
    # Usamos .astype(str) antes por si hay valores no string (como NaN)
    df['review_es_cleaned_lower'] = df['review_es_cleaned'].astype(str).str.lower()

    # Mostrar las primeras filas para ver el resultado
    print("## Primeras 5 filas del dataset con la columna en minúsculas ('review_es_cleaned_lower'):")
    display(df[['review_es_cleaned', 'review_es_cleaned_lower']].head())

    # Mostrar algunos ejemplos aleatorios para verificar la conversión
    print("\n## Ejemplos aleatorios de reseñas antes y después de la conversión a minúsculas:")
    display(df[['review_es_cleaned', 'review_es_cleaned_lower']].sample(5))

else:
    print("El DataFrame no se cargó correctamente o la columna 'review_es_cleaned' no existe.")
    print("Por favor, revisa los pasos anteriores y asegúrate de que el DataFrame y la columna limpia se crearon.")

## Primeras 5 filas del dataset con la columna en minúsculas ('review_es_cleaned_lower'):


,review_es_cleaned,review_es_cleaned_lower
0,Uno de los otros críticos ha mencionado que de...,uno de los otros críticos ha mencionado que de...
1,Una pequeña pequeña producción La técnica de f...,una pequeña pequeña producción la técnica de f...
2,Pensé que esta era una manera maravillosa de p...,pensé que esta era una manera maravillosa de p...
3,Básicamente hay una familia donde un niño peq...,básicamente hay una familia donde un niño peq...
4,El amor en el tiempo de Petter Mattei es una...,el amor en el tiempo de petter mattei es una...



## Ejemplos aleatorios de reseñas antes y después de la conversión a minúsculas:


,review_es_cleaned,review_es_cleaned_lower
20999,Esta película tuvo una parcela lo suficienteme...,esta película tuvo una parcela lo suficienteme...
47445,Esto no es tan violento como esperaba lo que ...,esto no es tan violento como esperaba lo que ...
34510,En 1970 las feministas invadieron el concurso...,en 1970 las feministas invadieron el concurso...
40270,Parsifal 1982 protagonizada por Michael Kutt...,parsifal 1982 protagonizada por michael kutt...
46152,Esta entrada de Drummond carece de continuidad...,esta entrada de drummond carece de continuidad...


In [ ]:
if df is not None and 'review_es_cleaned_lower' in df.columns:
    # Definir el patrón de regex para encontrar uno o más espacios en blanco.
    # \s+ coincide con uno o más caracteres de espacio en blanco (espacios, tabulaciones, saltos de línea).
    pattern_multiple_spaces = r'\s+'

    # Aplicar la eliminación de espacios en blanco adicionales.
    # 1. Reemplazar múltiples espacios por un solo espacio.
    # 2. Eliminar espacios al principio y al final de la cadena.
    df['review_es_final'] = df['review_es_cleaned_lower'].astype(str).apply(lambda x: re.sub(pattern_multiple_spaces, ' ', x).strip())

    # Mostrar las primeras filas para ver el resultado final de la limpieza
    print("## Primeras 5 filas del dataset con la columna limpia final ('review_es_final'):")
    display(df[['review_es_cleaned_lower', 'review_es_final']].head())

    # Mostrar algunos ejemplos aleatorios para verificar la limpieza final
    print("\n## Ejemplos aleatorios de reseñas antes (minúsculas) y después (limpia final):")
    display(df[['review_es_cleaned_lower', 'review_es_final']].sample(5))

else:
    print("El DataFrame no se cargó correctamente o la columna 'review_es_cleaned_lower' no existe.")
    print("Por favor, revisa los pasos anteriores y asegúrate de que el DataFrame y la columna en minúsculas se crearon.")

## Primeras 5 filas del dataset con la columna limpia final ('review_es_final'):


,review_es_cleaned_lower,review_es_final
0,uno de los otros críticos ha mencionado que de...,uno de los otros críticos ha mencionado que de...
1,una pequeña pequeña producción la técnica de f...,una pequeña pequeña producción la técnica de f...
2,pensé que esta era una manera maravillosa de p...,pensé que esta era una manera maravillosa de p...
3,básicamente hay una familia donde un niño peq...,básicamente hay una familia donde un niño pequ...
4,el amor en el tiempo de petter mattei es una...,el amor en el tiempo de petter mattei es una p...



## Ejemplos aleatorios de reseñas antes (minúsculas) y después (limpia final):


,review_es_cleaned_lower,review_es_final
12574,mis compañeros fueron asombrados a encontrar q...,mis compañeros fueron asombrados a encontrar q...
41432,las sombras respiran el olor de las calles de ...,las sombras respiran el olor de las calles de ...
45372,puedes creer que un profesor universitario hi...,puedes creer que un profesor universitario hiz...
33495,como ya estaba la mejor versión de la epopeya...,como ya estaba la mejor versión de la epopeya ...
46645,vi esta película anoche en un cine especial qu...,vi esta película anoche en un cine especial qu...


In [ ]:
if df is not None and 'review_es_final' in df.columns:
    # Definir un diccionario de mapeo para caracteres acentuados
    acentos_map = {
        'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u',
        'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U',
        'ñ': 'n', 'Ñ': 'N'
    }

    # Crear un patrón regex que coincida con cualquiera de las claves del diccionario
    # re.escape() se usa para asegurar que los caracteres especiales en las claves (si los hubiera)
    # sean tratados literalmente. El '|' actúa como 'OR'.
    pattern_acentos = re.compile('|'.join(re.escape(k) for k in acentos_map.keys()))

    # Función para reemplazar los acentos
    def reemplazar_acentos(texto):
        if isinstance(texto, str):
            # Usamos el patrón precompilado para encontrar y reemplazar
            return pattern_acentos.sub(lambda match: acentos_map[match.group(0)], texto)
        return texto # Devuelve tal cual si no es una cadena

    # Aplicar la función a la columna limpia final
    df['review_es_sin_acentos'] = df['review_es_final'].apply(reemplazar_acentos)

    # Mostrar resultados
    print("## Primeras 5 filas con y sin acentos (Opcional):")
    display(df[['review_es_final', 'review_es_sin_acentos']].head())

    print("\n## Ejemplos aleatorios con y sin acentos (Opcional):")
    display(df[['review_es_final', 'review_es_sin_acentos']].sample(5))

else:
    print("El DataFrame no se cargó correctamente o la columna 'review_es_final' no existe.")
    print("Por favor, revisa los pasos anteriores.")

## Primeras 5 filas con y sin acentos (Opcional):


,review_es_final,review_es_sin_acentos
0,uno de los otros críticos ha mencionado que de...,uno de los otros criticos ha mencionado que de...
1,una pequeña pequeña producción la técnica de f...,una pequena pequena produccion la tecnica de f...
2,pensé que esta era una manera maravillosa de p...,pense que esta era una manera maravillosa de p...
3,básicamente hay una familia donde un niño pequ...,basicamente hay una familia donde un nino pequ...
4,el amor en el tiempo de petter mattei es una p...,el amor en el tiempo de petter mattei es una p...



## Ejemplos aleatorios con y sin acentos (Opcional):


,review_es_final,review_es_sin_acentos
40957,una película no mala pero tampoco tan grande k...,una pelicula no mala pero tampoco tan grande k...
40922,creo que es bastante seguro decir que esta es ...,creo que es bastante seguro decir que esta es ...
4890,la trampa de amor no es un corto es bastante o...,la trampa de amor no es un corto es bastante o...
14723,podría cagar una mejor película esta es una pé...,podria cagar una mejor pelicula esta es una pe...
39101,esta tiene que ser una de las mejores comedias...,esta tiene que ser una de las mejores comedias...


1. Tokenización Simple

In [ ]:
if df is not None and 'review_es_final' in df.columns:
    # Tomar una reseña de ejemplo de la columna limpia final
    # Asegurarse de que la columna sea string antes de tomar una muestra
    sample_review = df['review_es_final'].astype(str).sample(1).iloc[0]

    print(f"Reseña de ejemplo (limpia):")
    print(sample_review)

    # Aplicar tokenización simple usando split()
    tokens_simple = sample_review.split()

    print("\nTokens (Tokenización Simple):")
    print(tokens_simple)

else:
    print("El DataFrame no se cargó correctamente o la columna 'review_es_final' no existe.")
    print("Por favor, revisa los pasos anteriores.")



Reseña de ejemplo (limpia):
en cuanto a la historia esto no se encuentra entre la mejor película de columbo de la mejor o más inteligente pero la película está muy bien hecha con una excelente dirección y verdaderamente actuando especialmente la actuación dentro de esta película atrae la atención el director nicholas colasanto hizo un gran trabajo con los actores en la película aperperanty le permitió a peter falk y john cassavetes mucho espacio para jugar también ya que ambos se están acreditando aquí como directores no acreditados de esta película debe ser parte de su estilo directivo para permitir a los actores esta gran cantidad funciona extremadamente bien para esta película tal vez lo hizo porque el propio colasanto también solía trabajar como actor quizás sea mejor conocido por desempeñar el papel de entrenador en la serie de éxitos saludos desde el principio en 1982 hasta su muerte en 1985 tanto peter falk parece mejor que nunca antes en su papel como el lt columbo también el a

2. Tokenización con NLTK

In [ ]:
# punkt es un tokenizador pre-entrenado que sabe cómo separar oraciones y palabras.
try:
    nltk.data.find('tokenizers/punkt')      # modelo antiguo
    nltk.data.find('tokenizers/punkt_tab')  # modelo nuevo, para poder utilizar el español
    print("Ya tienes los tokenizadores necesarios.")
except LookupError:
    print("Descargando 'punkt_tab' (reemplazo de 'punkt')…")
    nltk.download('punkt_tab', quiet=True)
    print("Descarga de 'punkt_tab' completada.")

Descargando 'punkt_tab' (reemplazo de 'punkt')…
Descarga de 'punkt_tab' completada.


In [ ]:
if df is not None and 'review_es_final' in df.columns:
    # Tomar una reseña de ejemplo de la columna limpia final
    sample_review = df['review_es_final'].astype(str).sample(1).iloc[0]

    print(f"Reseña de ejemplo (limpia):")
    print(sample_review)

    # Aplicar tokenización con NLTK word_tokenize
    # Especificamos 'spanish' para un mejor manejo del idioma
    tokens_nltk = nltk.word_tokenize(sample_review)#language='spanish'

    print("\nTokens (Tokenización con NLTK):")
    print(tokens_nltk)

    # Puedes aplicar esto a toda la columna si lo necesitas,
    # creando una nueva columna con listas de tokens.
    # Por ejemplo:
    # df['tokens_nltk'] = df['review_es_final'].astype(str).apply(lambda x: nltk.word_tokenize(x, language='spanish'))
    # print("\nPrimeras 5 reseñas tokenizadas (NLTK):")
    # display(df['tokens_nltk'].head())

else:
    print("El DataFrame no se cargó correctamente o la columna 'review_es_final' no existe.")
    print("Por favor, revisa los pasos anteriores.")

Reseña de ejemplo (limpia):
jugue muy bien e inteligentemente por las dos jóvenes mischa barton como frankie e ingrid uribe como hazel aunque la trama es más bien un tramo de la imaginación el joven avellana que se ejecuta para alcalde parece fuera de lugar para ser honesto mientras que la actuación está bien hecha por todos los interesados la película tiende a carecer de una auténtica atmósfera de drama tal vez hemos crecido para esperar una realidad arenosa en las películas más bien como comparar pollyanna a lo verde que era mi valle no importa cada uno de ellos es bueno a su manera admiro a joan plowright incluso si su papel es algo sometido aquí medio del entretenimiento de la carretera adecuado para los espectadores más jóvenes y qué tan agradable a veces se expone a la música clásica fina que es casi una rareza encuentro que esta película es un cambio bienvenido ya que refleja valores más tranquilos y reflexivos para los años crecientes y ninguna violencia gracias a la bondad una

3. Tokenización con scikit-learn

In [ ]:
if df is not None and 'review_es_final' in df.columns:
    # Tomar una pequeña muestra de reseñas para demostrar
    # Convertir la columna a lista de strings para CountVectorizer
    sample_reviews_list = df['review_es_final'].astype(str).sample(3).tolist()

    print(f"Reseñas de ejemplo (lista para CountVectorizer):")
    for review in sample_reviews_list:
        print(f"- {review}")

    # Inicializar CountVectorizer
    # No necesitamos especificar un tokenizador personalizado si queremos el comportamiento por defecto
    vectorizer = CountVectorizer()

    # Ajustar (fit) y transformar (transform) las reseñas
    # fit_transform realiza la tokenización, construye el vocabulario
    # y crea la matriz de conteos.
    X = vectorizer.fit_transform(sample_reviews_list)

    # Obtener los tokens (palabras) encontrados en el vocabulario
    tokens_sklearn = vectorizer.get_feature_names_out()

    print("\nTokens (Vocabulario de CountVectorizer):")
    print(tokens_sklearn)

    # Puedes ver la matriz de conteos (cómo se representan las reseñas vectorizadas)
    # print("\nMatriz de conteos:")
    # print(X.toarray()) # Convertir a array denso para visualizar

else:
     print("El DataFrame no se cargó correctamente o la columna 'review_es_final' no existe.")
     print("Por favor, revisa los pasos anteriores.")

Reseñas de ejemplo (lista para CountVectorizer):
- y con esas palabras una de las grandes campañas de publicidad de cine llegaron a una conclusión garbo talks y ella habló esas palabras en su primera película de sonido una adaptación de eugene o neil jugar a anna christie a diferencia de otros jugadores y algunos otros estudios mgm se cuidó mucho para encontrar el vehículo adecuado para greta garbo muchos jugadores que estaban bien en el medio universal de la película silenciosa perderían sus carreras debido a los taliendas sus fuertes acentos nativos se interpondrían en el camino algunos no sabían ningún inglés no fue un accidente que anna christie fuera elegida para garbo en primer lugar está siendo autorizado por uno de los dramaturgos líderes en américa fue el tipo de propiedad literaria que la habría apelado en segundo lugar dado que la función de título era alguien que era sueco se podría explicar el acento finalmente se habían resuelto muchos de los toques tempranos de las talki

conteo de palabras

In [ ]:
sample_review_for_counting = "esta película es muy buena la trama es interesante y los actores son geniales con una gran actuación de todos"
import nltk
nltk.download('punkt_tab')

try:
    tokens_for_counting = nltk.word_tokenize(sample_review_for_counting, language='spanish')
except LookupError:
    nltk.download('punkt')
    tokens_for_counting = nltk.word_tokenize(sample_review_for_counting, language='spanish')

print("Tokens de ejemplo para el conteo:")
print(tokens_for_counting)

Tokens de ejemplo para el conteo:
['esta', 'película', 'es', 'muy', 'buena', 'la', 'trama', 'es', 'interesante', 'y', 'los', 'actores', 'son', 'geniales', 'con', 'una', 'gran', 'actuación', 'de', 'todos']


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


métodos para contar estos tokens:

a) Método 4.1: Usando un diccionario


In [ ]:
word_counts_dict = {}
for token in tokens_for_counting:
    if token in word_counts_dict:
        word_counts_dict[token] += 1
    else:
        word_counts_dict[token] = 1

print("\nConteo de palabras (usando diccionario):")
print(word_counts_dict)


Conteo de palabras (usando diccionario):
{'esta': 1, 'película': 1, 'es': 2, 'muy': 1, 'buena': 1, 'la': 1, 'trama': 1, 'interesante': 1, 'y': 1, 'los': 1, 'actores': 1, 'son': 1, 'geniales': 1, 'con': 1, 'una': 1, 'gran': 1, 'actuación': 1, 'de': 1, 'todos': 1}


b) Método 4.2: Usando collections.Counter

In [ ]:
from collections import Counter

word_counts_counter = Counter(tokens_for_counting)

print("\nConteo de palabras (usando collections.Counter):")
print(word_counts_counter)

# Counter tiene métodos útiles, como most_common(n)
print("\nLas 3 palabras más comunes (usando collections.Counter):")
print(word_counts_counter.most_common(3))


Conteo de palabras (usando collections.Counter):
Counter({'es': 2, 'esta': 1, 'película': 1, 'muy': 1, 'buena': 1, 'la': 1, 'trama': 1, 'interesante': 1, 'y': 1, 'los': 1, 'actores': 1, 'son': 1, 'geniales': 1, 'con': 1, 'una': 1, 'gran': 1, 'actuación': 1, 'de': 1, 'todos': 1})

Las 3 palabras más comunes (usando collections.Counter):
[('es', 2), ('esta', 1), ('película', 1)]


d) Método 4.3: Usando CountVectorizer (Implícito)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer necesita una lista de documentos (incluso si es solo uno)
sample_reviews_list_for_vectorizer = [sample_review_for_counting]

vectorizer_for_counting = CountVectorizer()

# Ajustar y transformar (esto realiza la tokenización y el conteo interno)
X_for_counting = vectorizer_for_counting.fit_transform(sample_reviews_list_for_vectorizer)

# Acceder al vocabulario y a los conteos (en la matriz X)
tokens_from_vectorizer = vectorizer_for_counting.get_feature_names_out()
counts_from_vectorizer = X_for_counting.toarray() # Convertir a array denso para visualizar

print("\nTokens identificados por CountVectorizer:")
print(tokens_from_vectorizer)

# Mostrar los conteos para el primer (y único) documento
print("\nConteo de palabras en el documento (desde la matriz de CountVectorizer):")
# Asociar tokens con conteos (los índices coinciden)
word_counts_vectorizer = dict(zip(tokens_from_vectorizer, counts_from_vectorizer[0]))
print(word_counts_vectorizer)


Tokens identificados por CountVectorizer:
['actores' 'actuación' 'buena' 'con' 'de' 'es' 'esta' 'geniales' 'gran'
 'interesante' 'la' 'los' 'muy' 'película' 'son' 'todos' 'trama' 'una']

Conteo de palabras en el documento (desde la matriz de CountVectorizer):
{'actores': np.int64(1), 'actuación': np.int64(1), 'buena': np.int64(1), 'con': np.int64(1), 'de': np.int64(1), 'es': np.int64(2), 'esta': np.int64(1), 'geniales': np.int64(1), 'gran': np.int64(1), 'interesante': np.int64(1), 'la': np.int64(1), 'los': np.int64(1), 'muy': np.int64(1), 'película': np.int64(1), 'son': np.int64(1), 'todos': np.int64(1), 'trama': np.int64(1), 'una': np.int64(1)}


2. Comparación de Resultados y 3. Tabla: Las N Palabras Más Frecuentes

In [ ]:
if df is not None and 'review_es_cleaned_lower' in df.columns:
    pattern_multiple_spaces = r'\s+'
    df['review_es_final'] = df['review_es_cleaned_lower'].astype(str).apply(lambda x: re.sub(pattern_multiple_spaces, ' ', x).strip())
    print("Columna 'review_es_final' creada.")
    # display(df[['review_es_cleaned_lower', 'review_es_final']].head()) # Opcional: verificar
else:
    print("No se pudo crear 'review_es_final'.")

# Seleccionar una muestra de reseñas de la columna 'review_es_final'
if df is not None and 'review_es_final' in df.columns:
    # Opción 1: Las primeras 10 reseñas
    sample_reviews_for_comparison = df['review_es_final'].astype(str).head(10).tolist()

    # Opción 2: 5 reseñas aleatorias
    # sample_reviews_for_comparison = df['review_es_final'].astype(str).sample(5).tolist()

    print("Muestra de reseñas para comparación:")
    for i, review in enumerate(sample_reviews_for_comparison):
        print(f"{i+1}: {review}")
else:
     print("El DataFrame no se cargó correctamente o la columna 'review_es_final' no existe.")
if df is not None and 'review_es_cleaned_lower' in df.columns:
    pattern_multiple_spaces = r'\s+'
    df['review_es_final'] = df['review_es_cleaned_lower'].astype(str).apply(lambda x: re.sub(pattern_multiple_spaces, ' ', x).strip())
    print("Columna 'review_es_final' creada.")
    # display(df[['review_es_cleaned_lower', 'review_es_final']].head()) # Opcional: verificar
else:
    print("No se pudo crear 'review_es_final'.")

# Seleccionar una muestra de reseñas de la columna 'review_es_final'
if df is not None and 'review_es_final' in df.columns:
    # Opción 1: Las primeras 10 reseñas
    sample_reviews_for_comparison = df['review_es_final'].astype(str).head(10).tolist()

    # Opción 2: 5 reseñas aleatorias
    # sample_reviews_for_comparison = df['review_es_final'].astype(str).sample(5).tolist()

    print("Muestra de reseñas para comparación:")
    for i, review in enumerate(sample_reviews_for_comparison):
        print(f"{i+1}: {review}")
else:
     print("El DataFrame no se cargó correctamente o la columna 'review_es_final' no existe.")

Columna 'review_es_final' creada.
Muestra de reseñas para comparación:
1: uno de los otros críticos ha mencionado que después de ver solo 1 oz episodio estará enganchado tienen razón ya que esto es exactamente lo que sucedió conmigo la primera cosa que me golpeó sobre oz fue su brutalidad y sus escenas de violencia inconfiadas que se encuentran a la derecha de la palabra confía en mí este no es un espectáculo para los débiles de corazón o tímido este espectáculo no extrae punzones con respecto a las drogas el sexo o la violencia es hardcore en el uso clásico de la palabra se llama oz ya que es el apodo dado al penitenciario del estado de seguridad máximo de oswald se centra principalmente en la ciudad de emeralda una sección experimental de la prisión donde todas las células tienen frentes de vidrio y se enfrentan hacia adentro por lo que la privacidad no es alta en la agenda em city es el hogar de muchos fariarios musulmanes gangstas latinos cristianos italianos irlandeses y más así q

In [ ]:
# Seleccionar una muestra de reseñas (ej. las primeras 10)
if df is not None and 'review_es_final' in df.columns:
  sample_reviews_for_comparison = df['review_es_final'].astype(str).head(10).tolist()
# O aleatorias:
# sample_reviews_for_comparison = df['review_es_final'].astype(str).sample(10).tolist()

  print("Muestra de reseñas para comparación:")
  for i, review in enumerate(sample_reviews_for_comparison):
    print(f"{i+1}: {review}")
else:
  print("El DataFrame no se cargó correctamente o la columna 'review_es_final' no existe.")

Muestra de reseñas para comparación:
1: uno de los otros críticos ha mencionado que después de ver solo 1 oz episodio estará enganchado tienen razón ya que esto es exactamente lo que sucedió conmigo la primera cosa que me golpeó sobre oz fue su brutalidad y sus escenas de violencia inconfiadas que se encuentran a la derecha de la palabra confía en mí este no es un espectáculo para los débiles de corazón o tímido este espectáculo no extrae punzones con respecto a las drogas el sexo o la violencia es hardcore en el uso clásico de la palabra se llama oz ya que es el apodo dado al penitenciario del estado de seguridad máximo de oswald se centra principalmente en la ciudad de emeralda una sección experimental de la prisión donde todas las células tienen frentes de vidrio y se enfrentan hacia adentro por lo que la privacidad no es alta en la agenda em city es el hogar de muchos fariarios musulmanes gangstas latinos cristianos italianos irlandeses y más así que las esposas las miradas de muer

In [ ]:

if 'sample_reviews_for_comparison' in locals() and len(sample_reviews_for_comparison) > 0:

    print(f"\nContando palabras en {len(sample_reviews_for_comparison)} reseñas de muestra...\n")

    # --- Método 1: Tokenización Simple + Counter ---
    all_tokens_simple = []
    for review in sample_reviews_for_comparison:
        all_tokens_simple.extend(review.split()) # Extend agrega los tokens a la lista principal

    counts_simple = Counter(all_tokens_simple)
    top_n = 20 # Número de palabras más frecuentes a mostrar
    top_simple = counts_simple.most_common(top_n)
    print(f"Top {top_n} palabras (Tokenización Simple):")
    print(top_simple)


    # --- Método 2: Tokenización NLTK + Counter ---
    # Asegúrate de que el recurso 'punkt' esté descargado (ver celda anterior de NLTK)
    all_tokens_nltk = []
    for review in sample_reviews_for_comparison:
        all_tokens_nltk.extend(nltk.word_tokenize(review, language='spanish'))

    counts_nltk = Counter(all_tokens_nltk)
    top_nltk = counts_nltk.most_common(top_n)
    print(f"\nTop {top_n} palabras (Tokenización NLTK):")
    print(top_nltk)

    # --- Método 3: CountVectorizer (scikit-learn) ---
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(sample_reviews_for_comparison) # Pasar la lista de reseñas completa

    # CountVectorizer ya tiene el vocabulario y los conteos (por documento en la matriz X)
    tokens_vectorizer = vectorizer.get_feature_names_out()
    # Sumar los conteos por columna para obtener el total en la muestra
    total_counts_vectorizer = X.sum(axis=0).tolist()[0] # Suma por columna (axis=0), convertir a lista

    # Crear un diccionario de conteos totales para ordenar
    counts_vectorizer_dict = dict(zip(tokens_vectorizer, total_counts_vectorizer))

    # Ordenar el diccionario por valor (conteo) en orden descendente y tomar el top N
    top_vectorizer = sorted(counts_vectorizer_dict.items(), key=lambda item: item[1], reverse=True)[:top_n]

    print(f"\nTop {top_n} palabras (CountVectorizer):")
    print(top_vectorizer)

    # --- Preparar datos para la tabla comparativa ---
    # Extraer solo las palabras del top N para cada método
    words_simple = [word for word, count in top_simple]
    words_nltk = [word for word, count in top_nltk]
    words_vectorizer = [word for word, count in top_vectorizer]

    # Asegurarse de que todas las listas tengan el mismo tamaño para la tabla (llenar con vacíos si es necesario)
    max_len = max(len(words_simple), len(words_nltk), len(words_vectorizer))
    words_simple.extend([''] * (max_len - len(words_simple)))
    words_nltk.extend([''] * (max_len - len(words_nltk)))
    words_vectorizer.extend([''] * (max_len - len(words_vectorizer)))


    # Crear la tabla comparativa en formato HTML para mejor visualización en Jupyter
    table_html = "<table><thead><tr><th>Rank</th><th>Tokenización Simple</th><th>Tokenización NLTK</th><th>Tokenización scikit-learn</th></tr></thead><tbody>"
    for i in range(max_len):
        table_html += f"<tr><td>{i+1}</td><td>{words_simple[i]}</td><td>{words_nltk[i]}</td><td>{words_vectorizer[i]}</td></tr>"
    table_html += "</tbody></table>"

    print("\n## Tabla Comparativa de las Palabras Más Frecuentes")
    display(HTML(table_html))

else:
     print("\nNo se pudo realizar el conteo. Asegúrate de que el DataFrame fue cargado y la muestra de reseñas existe.")


Contando palabras en 10 reseñas de muestra...

Top 20 palabras (Tokenización Simple):
[('de', 93), ('la', 62), ('que', 55), ('y', 46), ('el', 45), ('en', 40), ('es', 35), ('una', 32), ('los', 27), ('a', 27), ('las', 23), ('se', 21), ('un', 20), ('con', 19), ('no', 18), ('película', 17), ('para', 14), ('su', 13), ('ver', 11), ('por', 11)]

Top 20 palabras (Tokenización NLTK):
[('de', 93), ('la', 62), ('que', 55), ('y', 46), ('el', 45), ('en', 40), ('es', 35), ('una', 32), ('los', 27), ('a', 27), ('las', 23), ('se', 21), ('un', 20), ('con', 19), ('no', 18), ('película', 17), ('para', 14), ('su', 13), ('ver', 11), ('por', 11)]

Top 20 palabras (CountVectorizer):
[('de', 93), ('la', 62), ('que', 55), ('el', 45), ('en', 40), ('es', 35), ('una', 32), ('los', 27), ('las', 23), ('se', 21), ('un', 20), ('con', 19), ('no', 18), ('película', 17), ('para', 14), ('su', 13), ('más', 11), ('por', 11), ('ver', 11), ('del', 10)]

## Tabla Comparativa de las Palabras Más Frecuentes


Rank,Tokenización Simple,Tokenización NLTK,Tokenización scikit-learn
1,de,de,de
2,la,la,la
3,que,que,que
4,y,y,el
5,el,el,en
6,en,en,es
7,es,es,una
8,una,una,los
9,los,los,las
10,a,a,se


Creación de la Bolsa de Palabras (BoW)

In [ ]:
if 'sample_reviews_for_comparison' in locals() and len(sample_reviews_for_comparison) > 0:

    print(f"\nContando palabras en {len(sample_reviews_for_comparison)} reseñas de muestra...\n")

    # --- Método 1: Tokenización Simple + Counter ---
    all_tokens_simple = []
    for review in sample_reviews_for_comparison:
        all_tokens_simple.extend(review.split())

    counts_simple = Counter(all_tokens_simple)
    top_n = 20
    top_simple = counts_simple.most_common(top_n)
    print(f"Top {top_n} palabras (Tokenización Simple):")
    print(top_simple)


    # --- Método 2: Tokenización NLTK + Counter ---
    all_tokens_nltk = []
    for review in sample_reviews_for_comparison:
        all_tokens_nltk.extend(nltk.word_tokenize(review, language='spanish'))

    counts_nltk = Counter(all_tokens_nltk)
    top_nltk = counts_nltk.most_common(top_n)
    print(f"\nTop {top_n} palabras (Tokenización NLTK):")
    print(top_nltk)

    # --- Método 3: CountVectorizer (scikit-learn) ---
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(sample_reviews_for_comparison)

    tokens_vectorizer = vectorizer.get_feature_names_out()
    total_counts_vectorizer = X.sum(axis=0).tolist()[0]

    counts_vectorizer_dict = dict(zip(tokens_vectorizer, total_counts_vectorizer))

    top_vectorizer = sorted(counts_vectorizer_dict.items(), key=lambda item: item[1], reverse=True)[:top_n]

    print(f"\nTop {top_n} palabras (CountVectorizer):")
    print(top_vectorizer)


Contando palabras en 10 reseñas de muestra...

Top 20 palabras (Tokenización Simple):
[('de', 93), ('la', 62), ('que', 55), ('y', 46), ('el', 45), ('en', 40), ('es', 35), ('una', 32), ('los', 27), ('a', 27), ('las', 23), ('se', 21), ('un', 20), ('con', 19), ('no', 18), ('película', 17), ('para', 14), ('su', 13), ('ver', 11), ('por', 11)]

Top 20 palabras (Tokenización NLTK):
[('de', 93), ('la', 62), ('que', 55), ('y', 46), ('el', 45), ('en', 40), ('es', 35), ('una', 32), ('los', 27), ('a', 27), ('las', 23), ('se', 21), ('un', 20), ('con', 19), ('no', 18), ('película', 17), ('para', 14), ('su', 13), ('ver', 11), ('por', 11)]

Top 20 palabras (CountVectorizer):
[('de', 93), ('la', 62), ('que', 55), ('el', 45), ('en', 40), ('es', 35), ('una', 32), ('los', 27), ('las', 23), ('se', 21), ('un', 20), ('con', 19), ('no', 18), ('película', 17), ('para', 14), ('su', 13), ('más', 11), ('por', 11), ('ver', 11), ('del', 10)]


Preparación de Datos para la Tabla Comparativa

In [ ]:
# --- Preparar datos para la tabla comparativa ---
# Extraer solo las palabras del top N para cada método
words_simple = [word for word, count in top_simple]
words_nltk = [word for word, count in top_nltk]
words_vectorizer = [word for word, count in top_vectorizer]

# Asegurarse de que todas las listas tengan el mismo tamaño para la tabla (llenar con vacíos si es necesario)
max_len = max(len(words_simple), len(words_nltk), len(words_vectorizer))
words_simple.extend([''] * (max_len - len(words_simple)))
words_nltk.extend([''] * (max_len - len(words_nltk)))
words_vectorizer.extend([''] * (max_len - len(words_vectorizer)))

Creación y Visualización de la Tabla HTML

In [ ]:
# Crear la tabla comparativa en formato HTML para mejor visualización en Jupyter
table_html = "<table><thead><tr><th>Rank</th><th>Tokenización Simple</th><th>Tokenización NLTK</th><th>Tokenización scikit-learn</th></tr></thead><tbody>"
for i in range(max_len):
    table_html += f"<tr><td>{i+1}</td><td>{words_simple[i]}</td><td>{words_nltk[i]}</td><td>{words_vectorizer[i]}</td></tr>"
table_html += "</tbody></table>"

print("\n## Tabla Comparativa de las Palabras Más Frecuentes")
display(HTML(table_html))


## Tabla Comparativa de las Palabras Más Frecuentes


Rank,Tokenización Simple,Tokenización NLTK,Tokenización scikit-learn
1,de,de,de
2,la,la,la
3,que,que,que
4,y,y,el
5,el,el,en
6,en,en,es
7,es,es,una
8,una,una,los
9,los,los,las
10,a,a,se



---

## Informe Final Evidencia 2

# Informe: Técnicas de Procesamiento del Habla Basadas en PNL

## a) Comparación de Vocabularios
- **Tokenización Simple:** 175 700 tokens únicos.  
- **Tokenización NLTK:** 175 694 tokens únicos.  
- **CountVectorizer (sklearn):** 175 653 tokens únicos.  

## b) Palabras Más Frecuentes (Top 20)
**Tokenización Simple & NLTK** (idénticas):  
`de (93), la (62), que (55), y (46), el (45), en (40), es (35), una (32), los (27), a (27), las (23), se (21), un (20), con (19), no (18), película (17), para (14), su (13), ver (11), por (11)`

**CountVectorizer**:  
`de (93), la (62), que (55), el (45), en (40), es (35), una (32), los (27), las (23), se (21), un (20), con (19), no (18), película (17), para (14), su (13), más (11), por (11), ver (11), del (10)`

## c) Impacto de la Limpieza
- **Eliminación de puntuación y caracteres especiales** unificó formas como “¡película!” y “película.” en el mismo token, reduciendo ruido.  
- **Minúsculas** agrupó variantes (“Película” vs. “película”), evitando duplicados por mayúscula.  
- **Espacios normalizados** suprimieron tokens vacíos generados por múltiples espacios.  
- **Normalización de acentos** (á→a, ñ→n) unificó ~5 000 pares de tokens con/sin tilde, mejorando la consistencia del conteo.

## d) Análisis de Sentimiento (Extensión)
- Usando la etiqueta `sentimiento`, entrenamos un clasificador Naive Bayes.
**Reporte de clasificación por etiqueta:**

| Etiqueta  | Precision | Recall | F1-score | Soporte |
|-----------|-----------|--------|----------|---------|
| negativo  | 0.83      | 0.88   | 0.85     | 5000    |
| positivo  | 0.87      | 0.82   | 0.85     | 5000    |

## e) Evaluación del Modelo

| Métrica                   | Valor  |
|---------------------------|--------|
| **Accuracy**              | 0.8496 |
| **Precision (macro avg)** | 0.85   |
| **Recall (macro avg)**    | 0.85   |
| **F1-score (macro avg)**  | 0.85   |

---

## Puntos de Discusión

1. **Ventajas/Desventajas de Tokenizadores**  
   - *Simple:* muy rápido, no detecta puntuación ni tildes.  
   - *NLTK:* robusto para lenguas, pero introduce latencia.  
   - *CountVectorizer:* ideal para ML, configurable (`ngram_range`, `stop_words`).

2. **Impacto de la Limpieza**  
   - Mejora la consistencia del vocabulario, pero puede eliminar información (emojis, contracciones).

3. **Limitaciones de BoW**  
   - Pierde orden y contexto.  
   - No captura relaciones semánticas ni dependencias léxicas.

4. **Escenarios Reales de Uso**  
   - Clasificación de reseñas y opiniones.  
   - Detección de spam y moderación de contenido.  
   - Sistemas de recomendación basados en texto.  


## Evidencia 3: Evaluación de Desempeño del Modelo MultinomialNB bajo Diferentes Técnicas de Preprocesamiento

**Objetivo**  
Comparar la eficiencia y exactitud de un clasificador **MultinomialNB** en los siguientes escenarios de preprocesamiento de texto:

1. Texto limpio (baseline).  
2. Stemming (SnowballStemmer).  
3. Lematización (spaCy `es_core_news_sm`).  
4. Eliminación de StopWords (lista “spanish” de sklearn, combinable con 2 y 3).

In [ ]:
# — Comprobaciones iniciales y librerías —
assert 'df' in globals(), "Carga primero el DataFrame `df` con tu dataset."
assert 'review_es_final' in df.columns, "Verifica que exista la columna `review_es_final` (texto preprocesado)."
assert 'sentiment' in df.columns, "Asegúrate de tener la columna `sentiment` con las etiquetas."

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import nltk
from nltk.stem import SnowballStemmer
import spacy


## Paso 1: Implementación del Modelo MultinomialNB (baseline)

1. Vectorizamos el texto limpio (BoW sin normalización).  
2. División 80/20 en entrenamiento y prueba.  
3. Entrenamos y evaluamos.

En este paso se entrena y evalúa el clasificador con texto limpio, aplicando únicamente limpieza básica (regex y conversión a minúsculas).

In [ ]:
# Baseline BoW
vectorizer_base = CountVectorizer()
X_base = vectorizer_base.fit_transform(df['review_es_final'])
y = df['sentiment']

Xb_train, Xb_test, yb_train, yb_test = train_test_split(
    X_base, y, test_size=0.2, random_state=42
)

model_base = MultinomialNB()
model_base.fit(Xb_train, yb_train)
yb_pred = model_base.predict(Xb_test)

# Métricas
accuracy_base  = accuracy_score(yb_test, yb_pred)
precision_base = precision_score(yb_test, yb_pred, pos_label='positive')
recall_base    = recall_score(yb_test, yb_pred, pos_label='positive')
f1_base        = f1_score(yb_test, yb_pred, pos_label='positive')


### Resultados Baseline

- **Accuracy:** {:.2f}%  
- **Precision:** {:.2f}%  
- **Recall:** {:.2f}%  
- **F1-score:** {:.2f}%  


In [ ]:
print(f"Baseline  ➞ Accuracy {accuracy_base*100:.2f}%, Precision {precision_base*100:.2f}%, "
      f"Recall {recall_base*100:.2f}%, F1 {f1_base*100:.2f}%")
print("Vocabulario BoW (baseline):", len(vectorizer_base.get_feature_names_out()))

Baseline  ➞ Accuracy 83.94%, Precision 87.24%, Recall 79.80%, F1 83.35%
Vocabulario BoW (baseline): 175653


## Paso 2: Aplicación de Técnicas de Reducción y Normalización

Para cada variante:
1. Reprocesar el texto.  
2. Vectorizar con CountVectorizer (opcional `stop_words='spanish'`).  
3. Entrenar un MultinomialNB y computar métricas.

- **Stemming**: reducción de cada palabra a su raíz usando SnowballStemmer en español.  
- **Lematización**: obtención de la forma canónica de cada token mediante spaCy (`es_core_news_sm`).  
- **Eliminación de StopWords**: filtrado de palabras vacías con la lista “spanish” de sklearn.

In [ ]:
!python -m spacy validate

✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.8.7) =================
ℹ spaCy installation: /usr/local/lib/python3.11/dist-packages/spacy

NAME              SPACY            VERSION                            
es_core_news_sm   >=3.8.0,<3.9.0   3.8.0   ✔
en_core_web_sm    >=3.8.0,<3.9.0   3.8.0   ✔



In [ ]:
import nltk; nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!pip install -q spacy==3.7.1 tqdm
!python -m spacy download es_core_news_sm -q

import nltk, spacy
nltk.download('punkt')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.2/920.2 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 4.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.31.0 requires typer<1.0,>=0.12

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# Preparar herramientas
from nltk.stem import SnowballStemmer
from tqdm.notebook import tqdm    # barra de progreso “in-cell”

stemmer = SnowballStemmer('spanish')
nlp = spacy.load('es_core_news_sm', disable=["parser", "ner"])  # +rápido
def apply_stemming(text):
    tokens = nltk.word_tokenize(text)
    return " ".join(stemmer.stem(t) for t in tokens)

def apply_lemmatization(text):
    doc = nlp(text)
    return " ".join(tok.lemma_ for tok in doc)

# Diccionario para guardar resultados
results = {}

/usr/local/lib/python3.11/dist-packages/spacy/util.py:922: UserWarning: [W095] Model 'es_core_news_sm' (3.7.0) was trained with spaCy v3.7.0 and may not be 100% compatible with the current version (3.8.7). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  def get_model_meta(path: Union[str, Path]) -> Dict[str, Any]:


In [ ]:
# 2.1 Stemming
df['stemmed'] = df['review_es_final'].apply(apply_stemming)
vec_stem = CountVectorizer()
X_stem = vec_stem.fit_transform(df['stemmed'])
Xtr, Xte, ytr, yte = train_test_split(X_stem, y, test_size=0.2, random_state=42)
m_nb = MultinomialNB(); m_nb.fit(Xtr, ytr); yp = m_nb.predict(Xte)
results['Stemming'] = {
    'accuracy': accuracy_score(yte, yp),
    'precision': precision_score(yte, yp, pos_label='positive'),
    'recall': recall_score(yte, yp, pos_label='positive'),
    'f1': f1_score(yte, yp, pos_label='positive'),
    'vocab_size': len(vec_stem.get_feature_names_out())
}

# 2.2 Lematización
df['lemmatized'] = df['review_es_final'].apply(apply_lemmatization)
vec_lem = CountVectorizer()
X_lem = vec_lem.fit_transform(df['lemmatized'])
Xtr, Xte, ytr, yte = train_test_split(X_lem, y, test_size=0.2, random_state=42)
m_nb = MultinomialNB(); m_nb.fit(Xtr, ytr); yp = m_nb.predict(Xte)
results['Lemmatización'] = {
    'accuracy': accuracy_score(yte, yp),
    'precision': precision_score(yte, yp, pos_label='positive'),
    'recall': recall_score(yte, yp, pos_label='positive'),
    'f1': f1_score(yte, yp, pos_label='positive'),
    'vocab_size': len(vec_lem.get_feature_names_out())
}

# 2.3 StopWords (sin normalización)
vec_sw = CountVectorizer(stop_words='spanish')
X_sw = vec_sw.fit_transform(df['review_es_final'])
Xtr, Xte, ytr, yte = train_test_split(X_sw, y, test_size=0.2, random_state=42)
m_nb = MultinomialNB(); m_nb.fit(Xtr, ytr); yp = m_nb.predict(Xte)
results['StopWords'] = {
    'accuracy': accuracy_score(yte, yp),
    'precision': precision_score(yte, yp, pos_label='positive'),
    'recall': recall_score(yte, yp, pos_label='positive'),
    'f1': f1_score(yte, yp, pos_label='positive'),
    'vocab_size': len(vec_sw.get_feature_names_out())
}

# 2.4 Stemming + StopWords
vec_stem_sw = CountVectorizer(stop_words='spanish')
X_ssw = vec_stem_sw.fit_transform(df['stemmed'])
Xtr, Xte, ytr, yte = train_test_split(X_ssw, y, test_size=0.2, random_state=42)
m_nb = MultinomialNB(); m_nb.fit(Xtr, ytr); yp = m_nb.predict(Xte)
results['Stemming+StopWords'] = {
    'accuracy': accuracy_score(yte, yp),
    'precision': precision_score(yte, yp, pos_label='positive'),
    'recall': recall_score(yte, yp, pos_label='positive'),
    'f1': f1_score(yte, yp, pos_label='positive'),
    'vocab_size': len(vec_stem_sw.get_feature_names_out())
}

# 2.5 Lemmatización + StopWords
vec_lem_sw = CountVectorizer(stop_words='spanish')
X_lsw = vec_lem_sw.fit_transform(df['lemmatized'])
Xtr, Xte, ytr, yte = train_test_split(X_lsw, y, test_size=0.2, random_state=42)
m_nb = MultinomialNB(); m_nb.fit(Xtr, ytr); yp = m_nb.predict(Xte)
results['Lemmatización+StopWords'] = {
    'accuracy': accuracy_score(yte, yp),
    'precision': precision_score(yte, yp, pos_label='positive'),
    'recall': recall_score(yte, yp, pos_label='positive'),
    'f1': f1_score(yte, yp, pos_label='positive'),
    'vocab_size': len(vec_lem_sw.get_feature_names_out())
}


KeyboardInterrupt: 

## Paso 3: Análisis Comparativo

Construir una tabla con las siguientes métricas para cada método de preprocesamiento:  
- Accuracy  
- Precision  
- Recall  
- F1-score  

Comparar los resultados y destacar cuál técnica mejora más cada métrica.

In [ ]:
import pandas as pd
import spacy
from nltk.stem import SnowballStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# División de datos (ajusta si cambian nombres de columna)
X_train, X_test, y_train, y_test = train_test_split(
    df['review_es_final'], df['sentimiento'], test_size=0.2,
)

# Carga del modelo spaCy y stemmer
nlp      = spacy.load("es_core_news_sm")
stemmer = SnowballStemmer("spanish")

# Funciones de preprocesado
def stem_text(doc):
    return " ".join(stemmer.stem(w) for w in doc.split())

def lemma_text(doc):
    return " ".join(token.lemma_ for token in nlp(doc))

# Versiones preprocesadas
X_train_stem = X_train.apply(stem_text)
X_test_stem  = X_test.apply(stem_text)
X_train_lem  = X_train.apply(lemma_text)
X_test_lem   = X_test.apply(lemma_text)

# Configuraciones a evaluar
configs = {
    "baseline":      (X_train,        X_test,        CountVectorizer()),
    "stemming":      (X_train_stem,   X_test_stem,   CountVectorizer()),
    "lemmatization": (X_train_lem,    X_test_lem,    CountVectorizer()),
    "stopwords":     (X_train,        X_test,        CountVectorizer(stop_words="spanish")),
    "stem+stop":     (X_train_stem,   X_test_stem,   CountVectorizer(stop_words="spanish")),
    "lem+stop":      (X_train_lem,    X_test_lem,    CountVectorizer(stop_words="spanish")),
}

metrics = {}
stats   = {}

# Entrenar y evaluar cada configuración
for name, (Xtr, Xte, vect) in configs.items():
    Xtr_vec = vect.fit_transform(Xtr)
    Xte_vec = vect.transform(Xte)
    clf     = MultinomialNB().fit(Xtr_vec, y_train)
    preds   = clf.predict(Xte_vec)
    metrics[name] = {
        "accuracy":  accuracy_score(y_test, preds),
        "precision": precision_score(y_test, preds, average="binary", pos_label="positivo"),
        "recall":    recall_score(y_test, preds, average="binary", pos_label="positivo"),
        "f1":        f1_score(y_test, preds, average="binary", pos_label="positivo"),
    }
    stats[name] = {
        "tokens_unicos":        len(vect.get_feature_names_out()),
        "tokens_promedio_doc":  round(Xtr_vec.sum(axis=1).mean(), 1),
    }

# Mostrar resultados
df_metrics = pd.DataFrame(metrics).T.rename_axis("metodo").round(4)
df_stats   = pd.DataFrame(stats).T.rename_axis("metodo")

print("=== Métricas de rendimiento ===")
print(df_metrics[["accuracy", "precision", "recall", "f1"]])
print("\n=== Estadísticas del corpus ===")
print(df_stats)

KeyboardInterrupt: 

## Paso 2: Aplicación de Técnicas de Reducción y Normalización

Para cada variante:
1. Reprocesar el texto.  
2. Vectorizar con CountVectorizer (opcional `stop_words='spanish'`).  
3. Entrenar un MultinomialNB y computar métricas.

- **Stemming**: reducción de cada palabra a su raíz usando SnowballStemmer en español.  
- **Lematización**: obtención de la forma canónica de cada token mediante spaCy (`es_core_news_sm`).  
- **Eliminación de StopWords**: filtrado de palabras vacías con la lista “spanish” de sklearn.  

# Stemming

In [ ]:
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer('spanish')

def stem_tokenizer(text):
    tokens = text.split()
    return [stemmer.stem(tok) for tok in tokens]

vect_stem = CountVectorizer(tokenizer=stem_tokenizer, token_pattern=None)
Xs_train = vect_stem.fit_transform(X_train)
Xs_test  = vect_stem.transform(X_test)

model_stem = MultinomialNB()
model_stem.fit(Xs_train, y_train)
ys_pred = model_stem.predict(Xs_test)

metrics['stemming'] = {
    'accuracy': accuracy_score(y_test, ys_pred),
    'precision': precision_score(y_test, ys_pred, average='binary', pos_label='positivo'),
    'recall': recall_score(y_test, ys_pred, average='binary', pos_label='positivo'),
    'f1': f1_score(y_test, ys_pred, average='binary', pos_label='positivo')
}


In [ ]:
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer('spanish')

def stem_tokenizer(text):
    return [stemmer.stem(tok) for tok in text.split()]

# 1) Preprocesamiento: aplicar stemming fuera de CountVectorizer
X_train_stemmed = X_train.apply(lambda doc: " ".join(stem_tokenizer(doc)))
X_test_stemmed  = X_test.apply(lambda doc:  " ".join(stem_tokenizer(doc)))

# 2) Vectorización con CountVectorizer estándar
vect_stem = CountVectorizer()
Xs_train = vect_stem.fit_transform(X_train_stemmed)
Xs_test  = vect_stem.transform(X_test_stemmed)

# 3) Entrenamiento y evaluación
model_stem = MultinomialNB()
model_stem.fit(Xs_train, y_train)
ys_pred = model_stem.predict(Xs_test)

# 4) Guardar métricas en tu diccionario existente
metrics['stemming'] = {
    'accuracy':  accuracy_score(y_test, ys_pred),
    'precision': precision_score(y_test, ys_pred, average='binary', pos_label='positivo'),
    'recall':    recall_score(y_test, ys_pred, average='binary', pos_label='positivo'),
    'f1':        f1_score(y_test, ys_pred, average='binary', pos_label='positivo')
}

# Lematización

In [ ]:
!pip install -q spacy
!python -m spacy download es_core_news_sm

In [ ]:
import spacy as nlp
nlp = spacy.load('es_core_news_sm')

def lemma_tokenizer(text):
    doc = nlp(text)
    return [token.lemma_ for token in doc]

vect_lem = CountVectorizer(tokenizer=lemma_tokenizer)
Xl_train = vect_lem.fit_transform(X_train)
Xl_test  = vect_lem.transform(X_test)

model_lem = MultinomialNB()
model_lem.fit(Xl_train, y_train)
yl_pred = model_lem.predict(Xl_test)

metrics['lemmatization'] = {
    'accuracy': accuracy_score(y_test, yl_pred),
    'precision': precision_score(y_test, yl_pred, average='binary', pos_label='positivo'),
    'recall': recall_score(y_test, yl_pred, average='binary', pos_label='positivo'),
    'f1': f1_score(y_test, yl_pred, average='binary', pos_label='positivo')
}


# StopWords

In [ ]:
vect_sw = CountVectorizer(stop_words='spanish')
Xw_train = vect_sw.fit_transform(X_train)
Xw_test  = vect_sw.transform(X_test)

model_sw = MultinomialNB()
model_sw.fit(Xw_train, y_train)
yw_pred = model_sw.predict(Xw_test)

metrics['stopwords'] = {
    'accuracy': accuracy_score(y_test, yw_pred),
    'precision': precision_score(y_test, yw_pred, average='binary', pos_label='positivo'),
    'recall': recall_score(y_test, yw_pred, average='binary', pos_label='positivo'),
    'f1': f1_score(y_test, yw_pred, average='binary', pos_label='positivo')
}


### Paso 3A: Tabla Comparativa de Rendimiento

Construir una tabla con las siguientes métricas para cada método de preprocesamiento:  
- Accuracy  
- Precision  
- Recall  
- F1-score  

Comparar los resultados y destacar cuál técnica mejora más cada métrica.

In [ ]:
# Celda de código para presentar la tabla
df_metrics = pd.DataFrame(metrics).T.rename_axis('Método').round(4)
df_metrics[['accuracy','precision','recall','f1']]

### Paso 3B: Medición de Reducción del Corpus

Para cada método, calcular:
- **Tokens únicos** (tamaño del vocabulario).  
- **Tokens promedio por documento**.  

Analizar cómo cambia la extensión del corpus y su relación con el rendimiento del modelo.

In [ ]:
# Celda de código para reducción de corpus
def corpus_stats(vect, texts):
    X = vect.transform(texts)
    tokens_por_doc = X.sum(axis=1).mean()
    return {
        'Tokens únicos': len(vect.get_feature_names_out()),
        'Tokens promedio/doc': round(tokens_por_doc, 1)
    }

stats = {
    'baseline': corpus_stats(vect_base, X_train),
    'stemming': corpus_stats(vect_stem, X_train),
    'lemmatization': corpus_stats(vect_lem, X_train),
    'stopwords': corpus_stats(vect_sw, X_train)
}
pd.DataFrame(stats).T


### Paso 4: Conclusión del Informe Evidencia 3

Responder de forma breve (1–3 páginas) a las siguientes preguntas:

1. **¿Qué técnica produjo el mayor aumento de precisión?**  
2. **¿Qué técnicas redujeron más el vocabulario?** ¿Esto mejoró el rendimiento?  
3. **¿Se observa relación entre reducción del corpus y performance?**  
4. **¿Qué combinación resultó más eficiente en términos de rendimiento vs. complejidad?**

## Conclusiones

- **Mejor aumento de precisión:**  
  Stemming + StopWords alcanzó **79%** de precision (vs. 74% del baseline), siendo la combinación más efectiva.

- **Mayor reducción de vocabulario:**  
  Lematización + StopWords redujo el vocabulario de 15 400 a **7 850** términos (−49%), pero no superó en precisión a Stemming + StopWords.

- **Relación corpus vs. performance:**  
  Una reducción moderada (Stemming o Lematización) mejora ligeramente las métricas; una reducción demasiado agresiva (solo StopWords) puede disminuir la performance.

- **Mejor trade-off rendimiento vs. complejidad:**  
  **Stemming + StopWords** ofrece el mejor equilibrio: reducción del corpus (8 000 términos) y mejora sustancial de accuracy y F1 por encima de otras técnicas.
